In [ ]:
from google.cloud import translate_v3
import os
import pandas as pd
import numpy as np
import csv

## Google Cloud API 정보, working directory 입력

In [ ]:
project_id = "your_project_id" ## GCP project id 
glossary_id = "your_glossary_id" ## name you want to give this glossary resource
location = 'us-central1'
credential_file = "/path/to/your/credentials.json"
gcs_glossary_uri = 'gs://path/to/your/storage/your_glossary.csv'

working_dir = '/your/working/dir'
source_file = 'test_source.csv' # Files that need translation
output_file = 'test_output.csv' # output file name

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=credential_file

In [ ]:
def translate_text_with_glossary(
    text,
    project_id,
    glossary_id
):
    global translated_output

    client = translate_v3.TranslationServiceClient()
    parent = client.location_path(project_id, location)

    glossary = client.glossary_path(
        project_id, location, glossary_id  # The location of the glossary
    )

    glossary_config = translate_v3.types.TranslateTextGlossaryConfig(
        glossary=glossary)

    # Supported language codes: https://cloud.google.com/translate/docs/languages
    response = client.translate_text(
        contents=[text],
        source_language_code="ko",
        target_language_code="en",
        parent=parent,
        glossary_config=glossary_config,
    )
    
    for translation in response.glossary_translations:
        translated_output = translation.translated_text

In [ ]:
df = pd.read_csv("/".join([working_dir,source_file]))

In [ ]:
for index, row in df.iterrows():
    df.loc[index,"en"] = ''

for index, row in df.iterrows():
    translator = translate_text_with_glossary(row["ko"], project_id, glossary_id)
    
    row["en"] = translated_output

In [ ]:
df.to_csv("/".join([working_dir,output_file]), mode='w')